In [22]:
import numpy as np
import pandas as pd

In [23]:
data = pd.read_csv('data_banknote_authentication.csv')
data = np.array(data)
# print(data[0:5])

In [24]:
X, Y = data[:,0:-1].T, data[:,-1].reshape(-1,1).T
print(f'Input Shape: {X.shape}\nOutput Shape: {Y.shape}')

Input Shape: (4, 1371)
Output Shape: (1, 1371)


In [25]:
# Number of features
n = X.shape[0]
# Number of examples
m = X.shape[1]
print(f"Number of features: {n}\nNumber of examples: {m}")

Number of features: 4
Number of examples: 1371


In [26]:
# Sigmoid helper function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [27]:
# Initialize training parameters W and b
def initialize_params():
    W = np.random.randn(1,n)
    b = np.zeros((1,1))
    return W, b

In [28]:
# Forward pass
def forward_prop(X, W, b):
    z = W @ X + b
    y_hat = sigmoid(z)
    return y_hat

In [29]:
# Compute the cost (negative log likelihood)
def compute_cost(y_hat, Y):
    epsilon = 1e-8
    J = np.sum(- (Y * np.log(y_hat + epsilon)) - ((1 - Y) * np.log(1 - y_hat + epsilon)))
    return J

In [30]:
# Backprop pass
def backprop(Y, y_hat, X):
    dJ__dy_hat = (-Y / y_hat) + (1 - Y)/(1 - y_hat)
    dy_hat__dz = y_hat * (1 - y_hat)
    dz__dw = X
    dz__db = 1

    dJ__dw = (dJ__dy_hat * dy_hat__dz) @ dz__dw.T
    dJ__db = (dJ__dy_hat * dy_hat__dz) # * dz__db

    return dJ__dw, dJ__db

In [31]:
# Update parameters
def update_params(W, b, dJdW, dJdb, alpha):
    W = W - alpha * dJdW
    b = b - alpha * dJdb
    return W, b

In [32]:
# Train the logistic regression classifier from pieces above
def logistic_regression(X, Y, num_iterations=1000, print_stage=100, alpha=0.01):

    W, b = initialize_params()

    for i in range(num_iterations + 1):

        y_hat = forward_prop(X, W, b)
        J = compute_cost(y_hat, Y)
        if i % print_stage == 0:
            print(f'Iter {i} cost: {J}')
        dJdW, dJdb = backprop(Y, y_hat, X)
        W, b = update_params(W, b, dJdW, dJdb, alpha)

    return W, b

In [33]:
W, b = logistic_regression(X, Y, alpha=0.0003)

Iter 0 cost: 475.1305235135174
Iter 100 cost: 139.2640018306359
Iter 200 cost: 134.24013307942312
Iter 300 cost: 131.60117084924025
Iter 400 cost: 129.73692376317285
Iter 500 cost: 128.20580222560827
Iter 600 cost: 126.83841855147709
Iter 700 cost: 125.56000732667445
Iter 800 cost: 124.33452037435943
Iter 900 cost: 123.14349453349658
Iter 1000 cost: 121.97705249680139
